# 프롬프트 관리 시스템 테스트

이 노트북은 `configs/prompts.yaml`과 `src/app/core/prompts.py`의 기능을 테스트합니다.

In [ ]:
import sys

sys.path.append("..")

import json

from src.app.core.prompts import build_messages, get_prompt, get_prompt_manager

## 1. PromptManager 초기화

In [ ]:
# PromptManager 인스턴스 가져오기
manager = get_prompt_manager()

print("✅ PromptManager 초기화 완료")
print(f"프롬프트 파일 경로: {manager.prompts_path}")

## 2. 카테고리 및 옵션 확인

In [ ]:
print("📋 사용 가능한 카테고리:")
for cat in manager.get_categories():
    print(f"  - {cat}")

print("\n📏 요약 길이 옵션:")
for length in manager.get_summary_lengths():
    print(f"  - {length}")

print("\n🏷️ 분류 카테고리:")
for cat in manager.get_classification_categories():
    print(f"  - {cat}")

print("\n🔬 연구 분야:")
for field in manager.get_research_fields():
    print(f"  - {field}")

## 3. 요약 프롬프트 테스트

In [ ]:
# 샘플 논문 데이터
sample_article = {
    "title": "Attention Is All You Need",
    "content": """
    We propose a new simple network architecture, the Transformer, 
    based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
    Experiments on two machine translation tasks show these models to be superior in quality 
    while being more parallelizable and requiring significantly less time to train.
    """,
}

print("=" * 60)
print("요약 프롬프트 테스트 (한국어 - Medium)")
print("=" * 60)

messages = manager.build_messages(
    "summarize", "korean.medium", title=sample_article["title"], content=sample_article["content"]
)

print("\n[System Prompt]")
print(messages[0]["content"])
print("\n[User Prompt]")
print(messages[1]["content"])

In [ ]:
# 다른 길이 옵션 테스트
print("=" * 60)
print("요약 프롬프트 테스트 (한국어 - Short)")
print("=" * 60)

messages_short = manager.build_messages(
    "summarize", "korean.short", title=sample_article["title"], content=sample_article["content"]
)

print("\n[User Prompt]")
print(messages_short[1]["content"])

## 4. 중요도 평가 프롬프트 테스트

In [ ]:
print("=" * 60)
print("중요도 평가 프롬프트 테스트")
print("=" * 60)

eval_messages = manager.build_messages(
    "evaluate_importance",
    title=sample_article["title"],
    content=sample_article["content"],
    metadata=json.dumps({"citations": 50000, "year": 2017}),
)

print("\n[System Prompt]")
print(eval_messages[0]["content"][:300], "...")
print("\n[User Prompt]")
print(eval_messages[1]["content"][:300], "...")

# 평가 기준 및 가중치
print("\n📊 평가 기준:")
criteria = manager.get_evaluation_criteria()
weights = manager.get_evaluation_weights()
for criterion in criteria:
    weight = weights.get(criterion, 0)
    print(f"  - {criterion}: {weight * 100}%")

## 5. 카테고리 분류 프롬프트 테스트

In [ ]:
print("=" * 60)
print("카테고리 분류 프롬프트 테스트")
print("=" * 60)

classify_messages = manager.build_messages(
    "classify_category",
    title=sample_article["title"],
    content=sample_article["content"],
    source_name="arXiv",
    url="https://arxiv.org/abs/1706.03762",
)

print("\n[System Prompt]")
print(classify_messages[0]["content"][:300], "...")
print("\n[User Prompt]")
print(classify_messages[1]["content"])

## 6. 프롬프트 포맷팅 테스트

In [ ]:
print("=" * 60)
print("프롬프트 포맷팅 테스트")
print("=" * 60)

template = manager.get("summarize.korean.medium.user_template")

# 변수 치환
formatted = manager.format_prompt(
    template, title="GPT-4 Technical Report", content="GPT-4는 대규모 멀티모달 모델입니다..."
)

print("\n[원본 템플릿]")
print(template[:200], "...")
print("\n[포맷된 결과]")
print(formatted[:200], "...")

## 7. 온보딩 프롬프트 테스트

In [ ]:
print("=" * 60)
print("온보딩 챗봇 프롬프트")
print("=" * 60)

# 환영 메시지
welcome = get_prompt("onboarding.welcome_message")
print("\n[환영 메시지]")
print(welcome)

# 질문 목록
print("\n[질문 목록]")
questions = get_prompt("onboarding.questions")
for key, value in questions.items():
    print(f"\n{key}:")
    print(f"  {value['prompt'][:100]}...")

## 8. 편의 함수 테스트

In [ ]:
print("=" * 60)
print("편의 함수 테스트")
print("=" * 60)

# get_prompt 함수
default_lang = get_prompt("common.default_language")
print(f"\n기본 언어: {default_lang}")

# build_messages 함수 (직접 호출)
messages = build_messages("summarize", "english.short", title="Test", content="Test content")
print(f"\n영어 요약 메시지 생성 성공: {len(messages)} messages")

# 에러 메시지
error_msgs = get_prompt("common.error_messages")
print("\n에러 메시지:")
for key, value in error_msgs.items():
    print(f"  {key}: {value}")

## 9. 종합 테스트

실제 사용 시나리오: 논문 하나를 받아서 모든 처리를 위한 프롬프트 생성

In [ ]:
print("=" * 60)
print("종합 테스트: 실제 사용 시나리오")
print("=" * 60)

# 실제 논문 데이터
article = {
    "title": "Language Models are Few-Shot Learners",
    "content": """
    We demonstrate that scaling up language models greatly improves task-agnostic, 
    few-shot performance. We train GPT-3, an autoregressive language model with 175 billion parameters,
    and test its performance in the few-shot setting. For all tasks, GPT-3 is applied without any 
    gradient updates or fine-tuning.
    """,
    "source_name": "arXiv",
    "url": "https://arxiv.org/abs/2005.14165",
    "metadata": {"citations": 15000, "year": 2020},
}

# 1. 요약 프롬프트
print("\n1️⃣ 요약 생성 프롬프트")
summary_msgs = build_messages(
    "summarize", "korean.medium", title=article["title"], content=article["content"]
)
print(f"✅ 메시지 생성 완료 ({len(summary_msgs)} messages)")

# 2. 중요도 평가 프롬프트
print("\n2️⃣ 중요도 평가 프롬프트")
eval_msgs = build_messages(
    "evaluate_importance",
    title=article["title"],
    content=article["content"],
    metadata=json.dumps(article["metadata"]),
)
print(f"✅ 메시지 생성 완료 ({len(eval_msgs)} messages)")

# 3. 카테고리 분류 프롬프트
print("\n3️⃣ 카테고리 분류 프롬프트")
classify_msgs = build_messages(
    "classify_category",
    title=article["title"],
    content=article["content"],
    source_name=article["source_name"],
    url=article["url"],
)
print(f"✅ 메시지 생성 완료 ({len(classify_msgs)} messages)")

print("\n" + "=" * 60)
print("✅ 모든 프롬프트 생성 성공!")
print("=" * 60)

## ✅ Checkpoint 1 확인

- [x] `configs/prompts.yaml` 존재
- [x] 프롬프트 로더 정상 작동
- [x] 프롬프트 템플릿 변수 치환 테스트 성공

**Step 1 완료!** 다음 Step 2로 진행할 수 있습니다.